<a href="https://colab.research.google.com/github/GbrlOl/whisper-fast/blob/main/Whisper_Large_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install --upgrade pip
!pip install --upgrade -qqq git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 61.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
sample = dataset[0]["audio"]

result = pipe(sample)
print(result["text"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating validation split:   0%|          | 0/1 [00:00<?, ? examples/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


 Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel. Nor is Mr. Quilter's manner less interesting than his matter. He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind. He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of rocky Ithaca. Linnell's pictures are a sort of Upguards and Adam paintings, and Mason's exquisite idylls are as national as a jingo poem. Mr. Burkett Foster's landscapes smile at one much in the same way that Mr. Carker used to flash his teeth. And Mr. John Collier gives his sitter a cheerful slap on the back before he says, like a shampooer in a Turkish bath, Next man!


In [3]:
result = pipe("/content/CLASE_9.mp3", generate_kwargs={"language": "spanish"})
result

{'text': ' bien, entonces ahora partió esto buenas tardes no sé si está grabando sí, buenas tardes estudiantes espero que estén súper bien vamos hoy día, como les comenté en la clase pasada, hoy día la idea de la clase de hoy es ver algún problema tipo prueba, para que ustedes sepan y entiendan qué es lo que estamos trabajando me imagino que ya han estudiado recuerden que en el aula virtual ya se les dejó el problema 1 de la prueba, así que no voy a tocar nada de eso hoy día, vamos a ver otras cosas que no sean de lo que les pregunto en esa pregunta, y eso primero, principalmente eso. Así que recordar que ya vimos la materia, ya tenemos la materia completa, recordar que ya vimos la materia ya tenemos la materia completa pasamos la toda la unidad, la clase pasada terminamos de ver la parte de TN y Norton y vimos también TLEGEN, potencia, hicimos problemas y no sé si ustedes tienen dudas o consultas hay 18 alumnos ahora espero que estén súper bien bien, entonces vamos a partir, abrí si u

In [4]:
result

{'text': ' bien, entonces ahora partió esto buenas tardes no sé si está grabando sí, buenas tardes estudiantes espero que estén súper bien vamos hoy día, como les comenté en la clase pasada, hoy día la idea de la clase de hoy es ver algún problema tipo prueba, para que ustedes sepan y entiendan qué es lo que estamos trabajando me imagino que ya han estudiado recuerden que en el aula virtual ya se les dejó el problema 1 de la prueba, así que no voy a tocar nada de eso hoy día, vamos a ver otras cosas que no sean de lo que les pregunto en esa pregunta, y eso primero, principalmente eso. Así que recordar que ya vimos la materia, ya tenemos la materia completa, recordar que ya vimos la materia ya tenemos la materia completa pasamos la toda la unidad, la clase pasada terminamos de ver la parte de TN y Norton y vimos también TLEGEN, potencia, hicimos problemas y no sé si ustedes tienen dudas o consultas hay 18 alumnos ahora espero que estén súper bien bien, entonces vamos a partir, abrí si u

In [5]:
type(result)

dict

In [ ]:
result

{'text': ' hola amigos como están acá perfecto nuevamente en youtube y vamos a aplicar la misma técnica que usé para las redes sociales para instalar para poder crecer ahí simplemente dar vídeos sencillos sin mucho color sin mucha edición ahí que vayan fluyendo de forma natural y hacerlo de forma muy constante y periódica como lo que usted tiene que hacer si es que quiere lograr algo bueno en su vida mejor hecho que perfecto me enseñó un buen amigo un buen dicho es y amigos yo hablando de cosas que son útiles de cosas que son importantes en la una de ellas es alimentarse de forma adecuada y acá también el dicho este de mejor hecho que perfecto aplica bien usted busque los alimentos bien naturales de origen natural de densidad nutricional alta prefiera los de esa manera se va a mantener siempre en forma con un buen porcentaje de grasa, con buena salud, con un buen balance de minerales de vitaminas y van a estar generalmente si usted ha hecho una miradita más o menos al plato, con videos